## $ \text{Customize Graph Convolution using Message Passing APIs} $

이전 세션 (GCN)에서는 GraphSAGE를 사용해서 Node와 Edge를 예측했습니다. 

이와 같은 방식은 Spectral Convoltion Network 이며, 이번 세션에서는 Message passing 즉, Spatial Convolution에 대해서 다룹니다. 

Message passing을 제안한 논문인 $ \text{Neural Message Passing for Quantum Chemistry}$ [Gilmer et al.](https://arxiv.org/pdf/1704.01212.pdf)는 GCN을 제안한 논문에서 사용한 방법을 모티브로 MPNN을 제안하였습니다. 

In [2]:
import dgl 

import torch 
import torch.nn as nn 
import torch.nn.functional as F 

import dgl.function as fn 

$$
h_{\mathcal{N}(v)}^k\leftarrow \text{AGGREGATE}_k\{h_u^{k-1},\forall u\in\mathcal{N}(v)\}
$$

$$
h_v^k\leftarrow \text{ReLU}\left(W^k\cdot \text{CONCAT}(h_v^{k-1}, h_{\mathcal{N}(v)}^k) \right)
$$

In [3]:
class SAGEConv(nn.Module):
    """
    Parameters
    ----------
    in_feat : int 
        Input feature size.
    out_feat : int
        Output features size.
    """
    def __init_(self, in_feat, out_feat):
        super(SAGEConv, self).__init__()
        self.linear = nn.Linear(in_feat*2, out_feat)
        
    def forward(self, g, h):
        """
        Parameters
        ----------
        g : Graph 
            The input graph.
        h : Tensor
            The input node features.
        """
        
        with g.local_scope():
            g.ndata['h'] = h 
            g.update_all(fn.copy_u('h', 'm'), fn.mean('m', 'h_neigh'))
            h_neigh = g.ndata['h_neigh']
            h_total = torch.cat([h, h_neigh], dim=1)
            return F.relu()

* Message Function `fn.copy_u('h', 'm')` `'h'`의 node feature를 복사해서 이웃 노드에게 전달하는 역할을 합니다. 
* Reduce Function `fn.mean('m', 'h_neigh')` 전달 받은 메세지(`'m'`)를 평균낸 후 `'h_neigh'`라는 이름으로 새로운 node feature를 저장합니다. 
* `update_all`은 모든 node와 edge에 대한 message function과 reduce function를 만들기 위한 함수입니다.


$$ 
m^{(l)}_{u~v} = M^{(l)} (h^{(l-1)}_v, h^{(l-1)}_u, e^{(l-1)}_{u~v}) 
$$ 

$$
m^{(l)}_v = \sum_{u \in \mathcal{N}(v)} m^{(l)}_{u~v}
$$

$$
h^{(l)}_v = U^{(l)}(h^{(l-1)}_v, m^{(l)}_v)
$$

$M^{(l)}$은 message function이며, $\sum$은 reduce function을 의미합니다. $U$는 node undate function을 의미합니다.

이를 통해 최종적으로 readout function $R$을 통과하여 $\hat{y}$를 예측하게 됩니다. 

$$
\hat{y} = R(\{ h^T_v | v \in G \})
$$

$h^T_v$는 T 번째 hidden state를 의미합니다. 이와 같은 과정을 3번 반복한 경우 $h^3_v$이 됩니다.

In [5]:
# user-defined message function that is equivalent to "fn.u_mul_e('h', 'w', 'm')"
def u_num_e_udf(edges):
    return {'m':edges.src['h'] * edges.data['w']}